# Deep Learning for Deciphering Traffic Signs
_________________________________________________________________________________________________________________________________________________________________________________

##### Contributors:
 Victor Floriano, Yifan Fan, Jose Salerno

## Problem Statement & Motivation
As the world advances towards autonomous vehicles, our team has observed the remarkable efforts of large car manufacturers, who are working with data scientists to develop fully autonomous cars. Our team is excited to contribute to the development of this technology by creating a neural network model that will be able to classify different traffic signs. Our ultimate goal is to assist car makers in overcoming the challenges they may face in implementing neural network models that effectively read traffic signs and further their efforts toward a fully autonomous car or assisted driving. We believe autonomous driving to be an important problem to solve due to the great economic benefits it can generate for car manufacturers and the improvement of general driving safety.

## Data Preparation
 We've selected the German Traffic Sign Recognition Benchmark (GTSRB) as our primary dataset. It's renowned for its complexity, featuring over 50,000 images across more than 40 classes of traffic signs. The GTSRB is publicly accessible through two resources. To efficiently manage the extensive and complex GTSRB dataset, our strategy integrates preprocessing for uniformity, data augmentation for robustness, and batch processing for computational efficiency. We'll employ distributed computing to parallelize operations, enhancing processing speed, and use stratified sampling for quick experimentation without compromising representativeness.



---





# Project Breakdown Summary
1. Data Cleaning and Transformation
3. MLP Models
4. CNN Model
5. Pre-trained Model
6. SVM
7. Results
8. Future Work
9. Conclusions

# 1. Data Cleaning and Transformation

## Importing Packages

In [2]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
use_gpu = True
gpu_available = torch.cuda.is_available()
gpu_available

True

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Transformation
train_datagenerator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    height_shift_range=0.2,
    validation_split=0.2
)

train_data = '/content/drive/MyDrive/BA865 - Group Project/GTSRBkaggle/Train'

In [6]:
train_generator = train_datagenerator.flow_from_directory(
    train_data,
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagenerator.flow_from_directory(
    train_data,
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 31376 images belonging to 43 classes.
Found 7843 images belonging to 43 classes.


In [7]:
# classes = { 0:'Speed limit (20km/h)',
#             1:'Speed limit (30km/h)',
#             2:'Speed limit (50km/h)',
#             3:'Speed limit (60km/h)',
#             4:'Speed limit (70km/h)',
#             5:'Speed limit (80km/h)',
#             6:'End of speed limit (80km/h)',
#             7:'Speed limit (100km/h)',
#             8:'Speed limit (120km/h)',
#             9:'No passing',
#             10:'No passing veh over 3.5 tons',
#             11:'Right-of-way at intersection',
#             12:'Priority road',
#             13:'Yield',
#             14:'Stop',
#             15:'No vehicles',
#             16:'Veh > 3.5 tons prohibited',
#             17:'No entry',
#             18:'General caution',
#             19:'Dangerous curve left',
#             20:'Dangerous curve right',
#             21:'Double curve',
#             22:'Bumpy road',
#             23:'Slippery road',
#             24:'Road narrows on the right',
#             25:'Road work',
#             26:'Traffic signals',
#             27:'Pedestrians',
#             28:'Children crossing',
#             29:'Bicycles crossing',
#             30:'Beware of ice/snow',
#             31:'Wild animals crossing',
#             32:'End speed + passing limits',
#             33:'Turn right ahead',
#             34:'Turn left ahead',
#             35:'Ahead only',
#             36:'Go straight or right',
#             37:'Go straight or left',
#             38:'Keep right',
#             39:'Keep left',
#             40:'Roundabout mandatory',
#             41:'End of no passing',
#             42:'End no passing veh > 3.5 tons' }

## Visualizing Data

In [8]:
train_generator.labels.shape

(31376,)

In [9]:
validation_generator.labels.shape

(7843,)

In [10]:
train_generator.class_mode

'categorical'

In [11]:
validation_generator.class_mode

'categorical'

In [12]:
train_generator.image_shape

(32, 32, 3)

In [13]:
validation_generator.image_shape

(32, 32, 3)

# 2. KERAS Sequential MLP


#### Sequential Model

In [15]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

def MLP():
    model = Sequential()
    model.add(Flatten(input_shape=(32, 32, 3)))
    model.add(Dense(256, activation='relu',))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(43, activation='softmax'))
    return model

In [16]:
model = MLP()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dense_1 (Dense)             (None, 256)               786688    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 43)               

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor='val_loss', patience=3)
model.fit(train_generator, validation_data = validation_generator, epochs = 3, callbacks = [callback])

Epoch 1/3
981/981 [==============================] - 3521s 4s/step - loss: 3.4247 - accuracy: 0.0774 - val_loss: 3.1777 - val_accuracy: 0.1258
Epoch 2/3
281/981 [=======>......................] - ETA: 1:34 - loss: 3.1697 - accuracy: 0.1098

## Sources:

- https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html